# The fonctions to import


In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('deep')



# The scraping of the funds of french private company's

In [2]:


def scrape_french_funds():
    url = 'https://www.investing.com/funds/france-funds'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        Name = []
        Symbol = []
        Last = []
        Chg = []
        Total_Assets = []
        Time = []

        table = soup.find('table')
        if table:
            rows = table.find_all('tr')[1:]  
            for row in rows:
                cols = row.find_all('td')
                # print(f"Columns in row: {[col.text.strip() for col in cols]}") # DEBUG PRINT: Show column contents
                if len(cols) >= 8:  
                    Name.append(cols[1].text.strip())  
                    Symbol.append(cols[2].text.strip()) 
                    Last.append(cols[3].text.strip())   
                    Total_Assets.append(cols[5].text.strip()) 
                    Time.append(cols[6].text.strip())   
                    try:
                        Chg_value = float(cols[4].text.strip().replace('%', '')) 
                        Chg.append(Chg_value)
                    except ValueError:
                        Chg.append(None)
                        print(f"Warning: Could not convert '{cols[4].text.strip()}' to float for 'Chg'")

        # Create DataFrame
        df = pd.DataFrame({
            'Name': Name,
            'Symbol': Symbol,
            'Last': Last,
            'Chg': Chg,
            'Total_Assets': Total_Assets,
            'Time': Time
            })

        return df

    except Exception as e:
        print(f'Error scraping data: {e}')
        return None

df = scrape_french_funds()
print(df)

                                         Name   Symbol         Last   Chg  \
0              Amundi Euro Liquidity SRI IC C  0P0000.  249,503.940  0.02   
1              Carmignac Patrimoine A EUR Acc  0P0000.      718.330 -0.58   
2             Carmignac Patrimoine A EUR Ydis  0P0000.      117.430 -0.58   
3              Carmignac Patrimoine E EUR Acc  0P0000.      172.700 -0.58   
4          Carmignac Patrimoine A CHF Acc Hdg  0P0000.      115.780 -0.60   
..                                        ...      ...          ...   ...   
95                               ALM Offensif  0P0000.      321.760 -0.09   
96              29 Haussmann Euro Rendement C  0P0000.    1,628.530 -0.61   
97  Candriam Index Arbitrage Classique en EUR  0P0000.    1,517.410  0.06   
98          BNP Paribas Actions Monde Classic  0P0000.       32.390 -1.10   
99   Natixis Ultra Short Term Bonds Plus SI C  0P0000.    1,711.400 -0.01   

   Total_Assets   Time  
0        55.37B  09/03  
1         6.16B  06/03  


# Fetch the data


In [3]:
funds_df = scrape_french_funds()

if funds_df is not None:
    print('French Companies Funds Data:')
    display(funds_df)

French Companies Funds Data:


,Name,Symbol,Last,Chg,Total_Assets,Time
0,Amundi Euro Liquidity SRI IC C,0P0000.,"249,503.940",0.02,55.37B,09/03
1,Carmignac Patrimoine A EUR Acc,0P0000.,718.330,-0.58,6.16B,06/03
2,Carmignac Patrimoine A EUR Ydis,0P0000.,117.430,-0.58,6.16B,06/03
3,Carmignac Patrimoine E EUR Acc,0P0000.,172.700,-0.58,6.16B,06/03
4,Carmignac Patrimoine A CHF Acc Hdg,0P0000.,115.780,-0.60,6.16B,06/03
...,...,...,...,...,...,...
95,ALM Offensif,0P0000.,321.760,-0.09,1.16B,05/03
96,29 Haussmann Euro Rendement C,0P0000.,"1,628.530",-0.61,1.31B,05/03
97,Candriam Index Arbitrage Classique en EUR,0P0000.,"1,517.410",0.06,257.12M,06/03
98,BNP Paribas Actions Monde Classic,0P0000.,32.390,-1.10,1.42B,05/03
